In [1]:
import pandas as pd 
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import rdflib
from rdflib import Graph, Namespace, URIRef, OWL, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD
from rdflib import RDF, RDFS, OWL, Namespace, Graph
import re

In [2]:
# Define the chunk size
chunk_size = 100000

# specify the file path
file_path = 'US_birth.csv'

# Initialize an empty list to store chunks
chunks = []

# read the file in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate the chunks to a single dataframe
df = pd.concat(chunks, ignore_index=True)
df.head()

C:\Users\mouad\AppData\Local\Temp\ipykernel_14768\3252050244.py:11: DtypeWarning: Columns (59,68,92,93,94,99) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_14768\3252050244.py:11: DtypeWarning: Columns (19,20,21,53,68,79,99,112) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_14768\3252050244.py:11: DtypeWarning: Columns (53,94) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipykernel_14768\3252050244.py:11: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
C:\Users\mouad\AppData\Local\Temp\ipyker

,Unnamed: 0,BirthMonth,TimeofBirth,BirthDayofWeek,BirthPlace,MotherSingleYearofAge,MotherAge,MotherNativity,ResidenceStatus,MotherRaceRecode31,...,CleftLipw/orw/oCleftPalate,CleftPalatealone,DownSyndrome,SuspectedChromosomalDisorder,Hypospadias,NoCongenitalAnomaliesChecked,InfantTransferred,InfantLivingatTimeofReport,InfantBreastfedatDischarge,SmokeStatus
0,0,April,2329,Tuesday,Hospital,33,30-34 Years,Born in the U.S. (50 US States),RESIDENT,White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
1,1,May,455,Friday,Hospital,28,25-29 Years,Born in the U.S. (50 US States),RESIDENT,White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
2,2,March,1357,Wednesday,Hospital,23,20-24 Years,Born in the U.S. (50 US States),RESIDENT,Black (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
3,3,April,1506,Wednesday,Hospital,22,20-24 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
4,4,October,2319,Wednesday,Hospital,19,15-19 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),...,No,No,No,No,No,True,No,Yes,Yes,Smoker


In [3]:
df= df.drop('Unnamed: 0',axis= 1)
df.head()

,BirthMonth,TimeofBirth,BirthDayofWeek,BirthPlace,MotherSingleYearofAge,MotherAge,MotherNativity,ResidenceStatus,MotherRaceRecode31,MotherRaceRecode6,...,CleftLipw/orw/oCleftPalate,CleftPalatealone,DownSyndrome,SuspectedChromosomalDisorder,Hypospadias,NoCongenitalAnomaliesChecked,InfantTransferred,InfantLivingatTimeofReport,InfantBreastfedatDischarge,SmokeStatus
0,April,2329,Tuesday,Hospital,33,30-34 Years,Born in the U.S. (50 US States),RESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
1,May,455,Friday,Hospital,28,25-29 Years,Born in the U.S. (50 US States),RESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
2,March,1357,Wednesday,Hospital,23,20-24 Years,Born in the U.S. (50 US States),RESIDENT,Black (only),Black (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
3,April,1506,Wednesday,Hospital,22,20-24 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Nonsmoker
4,October,2319,Wednesday,Hospital,19,15-19 Years,Born in the U.S. (50 US States),INTRASTATE NONRESIDENT,White (only),White (only),...,No,No,No,No,No,True,No,Yes,Yes,Smoker


In [4]:
# Load the ontology
g = Graph()
g.parse("C:/Users/mouad/Desktop/upwork/US Birth/data/BirthOntology.owl", format="xml")
selected_columns = []

# Get all object properties
object_properties = list(g.subjects(RDF.type, OWL.ObjectProperty))
for op in object_properties:
    print("Object Property:", op[op.find('#')+1:], end='\n \n')
    selected_columns.append(op[op.find('#')+4:])

# Get all data properties
data_properties = list(g.subjects(RDF.type, OWL.DatatypeProperty))
for dp in data_properties:
    print("Data Property:", dp[dp.find('#')+1:], end='\n \n')
    selected_columns.append(dp[dp.find('#')+1:])

Object Property: hasAdmissionToNICU
 
Object Property: hasAdmitToIntensiveCare
 
Object Property: hasAnencephaly
 
Object Property: hasAnesthesia
 
Object Property: hasAntibiotics
 
Object Property: hasAssistedVentilationOver6hrs
 
Object Property: hasAsstReproductiveTechnology
 
Object Property: hasAttendantAtBirth
 
Object Property: hasAugmentationOfLabor
 
Object Property: hasChlamydia
 
Object Property: hasChorioamnionitis
 
Object Property: hasCleftPalateAlone
 
Object Property: hasCongenitalDiaphragmaticHernia
 
Object Property: hasCyanoticCongenitalHeartDisease
 
Object Property: hasDeliveryMethodRecode1
 
Object Property: hasDeliveryStatus
 
Object Property: hasDownSyndrome
 
Object Property: hasFailedExternalCephalicVersion
 
Object Property: hasFatherEducation
 
Object Property: hasFatherHispanicOriginRecode
 
Object Property: hasFatherRaceRecode6
 
Object Property: hasFertilityEnhancingDrugs
 
Object Property: hasFetalPresentationAtDelivery
 
Object Property: hasFinalRouteAn

In [5]:
#get prefixes
g = Graph()
g.parse("C:/Users/mouad/Desktop/upwork/US Birth/data/BirthOntology.owl")

# Print all the namespaces defined in the graph
for prefix, uri in g.namespace_manager.namespaces():
    print(f"{prefix}: {uri}")

owl: http://www.w3.org/2002/07/owl#
rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
rdfs: http://www.w3.org/2000/01/rdf-schema#
xsd: http://www.w3.org/2001/XMLSchema#
xml: http://www.w3.org/XML/1998/namespace
: http://www.semanticweb.org/mypc/ontologies/2022/11/untitled-ontology-22#
orw: http://www.semanticweb.org/mypc/ontologies/2022/11/untitled-ontology-22#hasCleftLipw/orw/
swrl: http://www.w3.org/2003/11/swrl#
swrla: http://swrl.stanford.edu/ontologies/3.3/swrla.owl#
swrlb: http://www.w3.org/2003/11/swrlb#


In [6]:
rdf_df = pd.DataFrame({'subject': [], 'predicate': [], 'object': []})

In [7]:
h=0
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        rdf_df.loc[j+h,'subject']= 'BR'+str(i)
        rdf_df.loc[j+h,'predicate']= df.columns[j]
        rdf_df.loc[j+h,'object']= df.iloc[i,j]
    h+=116

KeyboardInterrupt: 

In [8]:
rdf_df

,subject,predicate,object
0,BR0,BirthMonth,April
1,BR0,TimeofBirth,2329
2,BR0,BirthDayofWeek,Tuesday
3,BR0,BirthPlace,Hospital
4,BR0,MotherSingleYearofAge,33
...,...,...,...
1403796,BR12101,MotherTransferred,No
1403797,BR12101,PaymentRecode,Medicaid
1403798,BR12101,FiveMinuteAPGARScore,09
1403799,BR12101,FiveMinuteAPGARRecode,A score of 9-10


In [14]:
rdf_df.to_csv('RDF-data.csv')